In [7]:
import httpx
import asyncio

LOGIN_URL = 'https://awm.infeo.at/api/getsecuretoken'
TRACKINGS_URL = 'https://awm.infeo.at/api/trackings/?startIndex=0&count=10&orderBy=-recorded&isExported=false&isExportedFlag=0&noExportFlag=0&isExportable=false&isValid=false&isInvalid=false&validityUnknown=false'

USERNAME = 'flr#hefel@infeo'
PASSWORD = 'Infeo_2023'
GRANT_TYPE = "password" 

async def get_access_token():
    async with httpx.AsyncClient() as client:
        
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": GRANT_TYPE
        }
        response = await client.post(LOGIN_URL, data=data, headers=headers)
        
        print(response)
        
        if response.status_code == 200:
            return response.json().get("access_token")  
        else:
            print(f"Failed to authenticate: {response.status_code}, {response.text}")
            return None

async def get_trackings(access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    
    async with httpx.AsyncClient() as client:
        response = await client.get(TRACKINGS_URL, headers=headers)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to get trackings: {response.status_code}, {response.text}")
            return None

async def main():
    token = await get_access_token()
    if token:
        trackings = await get_trackings(token)
        print(trackings)

await main()


<Response [200 OK]>
{'total': 101353, 'items': [{'id': 100914, 'lastChange': '2025-03-06T13:32:39+01:00', 'name': '1cdfc79edb634c5c8f5e084c2276b43f', 'description': '1359@awm_steiner', 'recorded': '2025-03-05T00:00:00+01:00', 'vehicleId': None, 'vehicleName': '', 'tourId': None, 'tourName': '', 'tourColor': None, 'length': 97.91008657336121, 'duration': 333170000000, 'isExported': 0, 'noExport': 0, 'isInvalid': None, 'editState': 0, 'minimumConnectionLength': 0, 'dumpZone': 0, 'externalId': None, 'points': [], 'metaData': []}, {'id': 100913, 'lastChange': '2025-03-06T13:32:39+01:00', 'name': '8aa43ce03bc54486875ef97f680c06e6', 'description': '1358@awm_steiner', 'recorded': '2025-03-05T00:00:00+01:00', 'vehicleId': None, 'vehicleName': '', 'tourId': None, 'tourName': '', 'tourColor': None, 'length': 134.66980425235172, 'duration': 385710000000, 'isExported': 0, 'noExport': 0, 'isInvalid': None, 'editState': 0, 'minimumConnectionLength': 0, 'dumpZone': 0, 'externalId': None, 'points': []